![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

# Havasu out-db Raster Example

In this notebook, we'll demonstrate how to load a large GeoTIFF file stored on S3 as out-db raster, and split it into smaller tiles.

We'll also show how to run RS_Value using a DataFrame of points on a large out-db raster. Read more about [Havasu](https://docs.wherobots.com/latest/references/havasu/introduction/), and [WherobotsDB Raster support](https://docs.wherobots.com/latest/references/havasu/raster/raster-overview/) in the documentation.

# Define Sedona context

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, lit
from sedona.spark import *

In [ ]:
config = SedonaContext.builder().appName('havasu-iceberg-outdb-raster-etl')\
    .config("spark.hadoop.fs.s3a.bucket.wherobots-examples.aws.credentials.provider","org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider")\
    .getOrCreate()
sedona = SedonaContext.create(config)

# Load Raster

We'll load the world population data, which contains estimated total number of people per grid-cell. The dataset is available to download in Geotiff format at a resolution of 30 arc (approximately 1km at the equator). The projection is Geographic Coordinate System, WGS84.

The original data can be retrieved from [here](https://hub.worldpop.org/geodata/summary?id=24777).

In [ ]:
raster_df = sedona.sql("SELECT RS_FromPath('s3://wherobots-examples/data/ppp_2020_1km_Aggregated.tif') as rast")
raster_df.show(5)

We can save this one large out-db raster as a Havasu table. The table will contain one row representing that large out-db raster.

In [ ]:
sedona.sql("CREATE NAMESPACE IF NOT EXISTS wherobots.test_db")
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.world_pop")
raster_df.writeTo("wherobots.test_db.world_pop").create()

In [ ]:
sedona.sql("SELECT RS_Metadata(rast) meta FROM wherobots.test_db.world_pop").show(5, False)

# Split raster into tiles

Large rasters may not be suitable for performing raster processing tasks that reads all the pixel data. WherobotsDB provides `RS_TileExplode` function for splitting the large raster into smaller tiles. When the input raster is an out-db raster, the generated tiles are out-db rasters referencing different parts of the out-db raster file. This is a pure geo-referencing metadata operation so this is very fast.

In [ ]:
tile_df = sedona.sql("SELECT RS_TileExplode(rast, 256, 256) AS (x, y, tile) FROM wherobots.test_db.world_pop")
tile_df.show(5)

## Saving as out-db rasters

In [ ]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.world_pop_tiles")
tile_df.writeTo("wherobots.test_db.world_pop_tiles").create()

In [ ]:
sedona.table("wherobots.test_db.world_pop_tiles").count()

## Saving tiles as in-db rasters

WherobotsDB provides an `RS_AsInDb` function for converting out-db raster as in-db raster. It needs to fetch all the band data from the raster file. We manually repartition the out-db tile dataset to run this convertion with high parallelism.

In [ ]:
indb_tile_df = tile_df.repartition(100).withColumn("tile", expr("RS_AsInDb(tile)"))
indb_tile_df.show(5)

In [ ]:
sedona.sql("DROP TABLE IF EXISTS wherobots.test_db.world_pop_indb_tiles")
indb_tile_df.writeTo("wherobots.test_db.world_pop_indb_tiles").create()

In [ ]:
sedona.table("wherobots.test_db.world_pop_indb_tiles").count()

## Visualize the tile boundaries on a map

In [ ]:
sedona.table("wherobots.test_db.world_pop_indb_tiles").show()
tiledMap = SedonaKepler.create_map()
SedonaKepler.add_df(tiledMap, sedona.table("wherobots.test_db.world_pop_indb_tiles").withColumn("tile", expr("RS_Envelope(tile)")), name="tiles")
tiledMap

# Population of POIs

We'll join the POI dataset with the population dataset to evaluate the population of POIs.

## Load POI Dataset

In [ ]:
spatialRdd = ShapefileReader.readToGeometryRDD(sedona.sparkContext, "s3://wherobots-examples/data/ne_50m_airports")
poi_df = Adapter.toDf(spatialRdd, sedona)
poi_df.show(5)

## Joining POIs with out-db raster

We can perform a catesian join with the single row large out-db raster table, and evaluates the population value on each point.

In [ ]:
res_df = poi_df.join(sedona.table("wherobots.test_db.world_pop")).withColumn("pop", expr("RS_Value(rast, geometry)")).drop("rast")
res_df.show(5)
res_df.where("pop > 100").count()

## Joining POIs with out-db tiles

We run a spatial join using the POI and out-db raster tile dataset, and evaluates the population value on each point.

In [ ]:
res_df = poi_df.join(sedona.table("wherobots.test_db.world_pop_tiles"), expr("RS_Intersects(tile, geometry)")).withColumn("pop", expr("RS_Value(tile, geometry)")).drop("tile")
res_df.show(5)
res_df.where("pop > 100").count()

## Joining POIs with in-db tiles

In [ ]:
res_df = poi_df.join(sedona.table("wherobots.test_db.world_pop_indb_tiles"), expr("RS_Intersects(tile, geometry)")).withColumn("pop", expr("RS_Value(tile, geometry)")).drop("tile")
res_df.show(5)
res_df.where("pop > 100").count()